In [1]:
# импортируем классы BaseEstimator и TransformerMixin,
# библиотеки pandas и numpy, модуль os
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np
import os

In [2]:
# взглянем на наш рабочий каталог
os.getcwd()

'/Users/artemgruzdev/Documents/Курс/Course_ML/Модуль_1'

In [3]:
# сменим рабочий каталог
os.chdir('/Users/artemgruzdev/Documents/Курс/Course_ML/Модуль_2')
# вернем обратно
os.chdir('/Users/artemgruzdev/Documents/Курс/Course_ML/Модуль_1')

In [4]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm.csv', sep=';')

In [5]:
# смотрим данные
data.head(3)

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
0,18975.456110,65999,237,1,14,0,6,0
1,4715.321344,0,65,19,56,0,3,0
2,5018.885233,54500,63,28,17,0,6,0


In [6]:
# импортируем функцию train_test_split(), с помощью
# которой разбиваем данные на обучающие и тестовые
from sklearn.model_selection import train_test_split
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(data.drop('Response', axis=1), 
                                                    data['Response'], 
                                                    test_size=0.3,
                                                    stratify=data['Response'],
                                                    random_state=42)

In [7]:
# создаем собственный класс, выполняющий замену
# пропусков средним значением
class MeanImputer(BaseEstimator, TransformerMixin):
    """
    Параметры:
    copy: возвращает копию
    """
    def __init__(self, copy=True):
        # все параметры для инициализации публичных атрибутов 
        # должны быть заданы в методе __init__
        
        # публичный атрибут
        self.copy = copy
        
    def __is_numpy(self, X):
        # частный метод, который с помощью функции isinstance()
        # проверяет, является ли наш объект массивом NumPy
        return isinstance(X, np.ndarray)
                
    def fit(self, X, y=None):
        # fit должен принимать в качестве аргументов X и y
        
        # обучение модели осуществляется прямо здесь
        # создаем пустой словарь, в котором ключами
        # будут имена/целые числа, а значениями - средние
        self._encoder_dict = {}
        
        # если 1D-массив, то переводим в 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
            
        # записываем количество столбцов
        ncols = X.shape[1]
        
        # записываем результат __is_numpy()
        is_np = self.__is_numpy(X)
        
        # если объект - массив NumPy, выполняем следующие действия:
        if is_np:
            # по каждому столбцу массива NumPy
            for col in range(ncols):
                # вычисляем среднее и записываем в словарь
                self._encoder_dict[col] = np.nanmean(X[:, col])
        # в противном случае, т.е. если объект - датафрейм pandas,
        # выполняем следующие действия:
        else:
            # по каждому столбцу датафрейма pandas
            for col in X.columns:
                # вычисляем среднее и записываем в словарь
                self._encoder_dict[col] = X[col].mean()

        # fit возвращает self
        return self
    
    def transform(self, X):
        # transform принимает в качестве аргумента только X
        
        # выполняем копирование массива во избежание предупреждения SettingWithCopyWarning
        # "A value is trying to be set on a copy of a slice from a DataFrame (Происходит 
        # попытка изменить значение в копии среза данных датафрейма)"
        if self.copy:
            X = X.copy()
        
        # если 1D-массив, то переводим в 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
            
        # записываем количество столбцов
        ncols = X.shape[1]
        
        # записываем результат __is_numpy()
        is_np = self.__is_numpy(X)
        
        # применяем преобразование к X
        # если объект - массив NumPy, выполняем следующие действия:
        if is_np:
            # по каждому столбцу массива NumPy
            for col in range(ncols):
                # заменяем пропуски средним значением из словаря
                X[:, col] = np.nan_to_num(X[:, col], nan=self._encoder_dict[col])
        # в противном случае, т.е. если объект - датафрейм pandas,
        # выполняем следующие действия:
        else:
            # по каждому столбцу датафрейма pandas
            for col in X.columns:
                # заменяем пропуски средним значением из словаря
                X[col] = np.where(X[col].isnull(), self._encoder_dict[col], X[col])
        
        # transform возвращает X
        return X

In [8]:
# создаем игрушечный обучающий датафрейм pandas
toy_train = pd.DataFrame({'Balance': [8.3, np.NaN, 10.2, 3.1], 
                        'Age': [23, 29, 36, np.NaN]})
toy_train

,Balance,Age
0,8.3,23.0
1,NaN,29.0
2,10.2,36.0
3,3.1,NaN


In [9]:
# создаем игрушечный тестовый датафрейм pandas
toy_test = pd.DataFrame({'Balance': [10.4, np.NaN, 22.5, 1.1], 
                         'Age': [13, 19, 66, np.NaN]})
toy_test

,Balance,Age
0,10.4,13.0
1,NaN,19.0
2,22.5,66.0
3,1.1,NaN


In [10]:
# смотрим, как будут выглядеть преобразования в игрушечных
# обучающем и тестовом датафреймах pandas
for col in toy_train.columns:
    toy_train[col].fillna(toy_train[col].mean(), inplace=True)
    toy_test[col].fillna(toy_train[col].mean(), inplace=True)
print('обучающий датафрейм')
print(toy_train)
print('')
print('тестовый датафрейм')
print(toy_test)

обучающий датафрейм
   Balance        Age
0      8.3  23.000000
1      7.2  29.000000
2     10.2  36.000000
3      3.1  29.333333

тестовый датафрейм
   Balance        Age
0     10.4  13.000000
1      7.2  19.000000
2     22.5  66.000000
3      1.1  29.333333


In [11]:
# создаем экземпляр класса MeanImputer
imp = MeanImputer()
# обучаем модель
imp.fit(toy_train)
# выполняем преобразование игрушечного
# обучающего датафрейма pandas
toy_train = imp.transform(toy_train)
toy_train

,Balance,Age
0,8.3,23.000000
1,7.2,29.000000
2,10.2,36.000000
3,3.1,29.333333


In [12]:
# выполняем преобразование игрушечного
# тестового датафрейма pandas
toy_test = imp.transform(toy_test)
toy_test

,Balance,Age
0,10.4,13.000000
1,7.2,19.000000
2,22.5,66.000000
3,1.1,29.333333


In [13]:
# создаем игрушечный обучающий датафрейм pandas
toy_train = pd.DataFrame({'Balance': [8.3, np.NaN, 10.2, 3.1], 
                          'Age': [23, 29, 36, np.NaN]})

# создаем экземпляр класса, отключив копирование
imp = MeanImputer(copy=False)
# обучаем модель
imp.fit(toy_train[['Age']])
# применяем модель
toy_train['Age'] = imp.transform(toy_train[['Age']])
toy_train

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Balance,Age
0,8.3,23.000000
1,NaN,29.000000
2,10.2,36.000000
3,3.1,29.333333


In [14]:
# создаем игрушечный обучающий датафрейм pandas
toy_train = pd.DataFrame({'Balance': [8.3, np.NaN, 10.2, 3.1], 
                          'Age': [23, 29, 36, np.NaN]})
# создаем экземпляр класса, отключив копирование
imp = MeanImputer(copy=True)
# обучаем модель
imp.fit(toy_train[['Age']])
# применяем модель
toy_train['Age'] = imp.transform(toy_train[['Age']])
toy_train

,Balance,Age
0,8.3,23.000000
1,NaN,29.000000
2,10.2,36.000000
3,3.1,29.333333


In [15]:
# создаем игрушечный обучающий массив NumPy
np_toy_train = np.array(pd.DataFrame({'Balance': [8.3, np.NaN, 10.2, 3.1], 
                                      'Age': [23, 29, 36, np.NaN]}))
np_toy_train

array([[ 8.3, 23. ],
       [ nan, 29. ],
       [10.2, 36. ],
       [ 3.1,  nan]])

In [16]:
# создаем игрушечный тестовый массив NumPy
np_toy_test = np.array(pd.DataFrame({'Balance': [10.4, np.NaN, 22.5, 1.1], 
                                     'Age': [13, 19, 66, np.NaN]}))
np_toy_test

array([[10.4, 13. ],
       [ nan, 19. ],
       [22.5, 66. ],
       [ 1.1,  nan]])

In [17]:
# обучаем модель
imp.fit(np_toy_train)
# выполняем преобразование игрушечного
# обучающего массива NumPy
np_toy_train = imp.transform(np_toy_train)
np_toy_train

array([[ 8.3       , 23.        ],
       [ 7.2       , 29.        ],
       [10.2       , 36.        ],
       [ 3.1       , 29.33333333]])

In [18]:
# выполняем преобразование игрушечного
# тестового массива NumPy
np_toy_test = imp.transform(np_toy_test)
np_toy_test

array([[10.4       , 13.        ],
       [ 7.2       , 19.        ],
       [22.5       , 66.        ],
       [ 1.1       , 29.33333333]])

In [19]:
# создаем 1-D массив NumPy
array = np.array([8.3, np.NaN, 10.2, 3.1])
array

array([ 8.3,  nan, 10.2,  3.1])

In [20]:
# проверяем размерность массива
array.ndim

1

In [21]:
# проверяем работу класса
imp.fit(array)
array = imp.transform(array)
array

array([[ 8.3],
       [ 7.2],
       [10.2],
       [ 3.1]])

In [22]:
# проверяем размерность массива
array.ndim

2

In [23]:
# создаем игрушечный датафрейм pandas
toy_train = pd.DataFrame({'Balance': [8.3, np.NaN, 10.2, 3.1], 
                          'Age': [23, 29, 36, np.NaN]})

# создаем пустой словарь encoder_dict
encoder_dict = {}

# по каждой переменной
for col in toy_train.columns:
    # печатаем имя
    print(col)
    # вычисляем среднее и записываем в словарь
    encoder_dict[col] = toy_train[col].mean()
    # печатаем словарь
    print(encoder_dict)
    print('')
    
# печатаем итоговый словарь
print('итоговый словарь', encoder_dict)

Balance
{'Balance': 7.2}

Age
{'Balance': 7.2, 'Age': 29.333333333333332}

итоговый словарь {'Balance': 7.2, 'Age': 29.333333333333332}


In [24]:
# создаем игрушечный обучающий массив NumPy
np_toy_train = np.array(pd.DataFrame({'Balance': [8.3, np.NaN, 10.2, 3.1], 
                                      'Age': [23, 29, 36, np.NaN]}))

# создаем пустой словарь encoder_dict
encoder_dict = {}

# по каждой переменной
for col in range(np_toy_train.shape[1]):
    # печатаем имя
    print(col)
    # вычисляем среднее и записываем в словарь
    encoder_dict[col] = np.nanmean(np_toy_train[:, col])
    # печатаем словарь
    print(encoder_dict)
    print('')
    
# печатаем итоговый словарь
print('итоговый словарь', encoder_dict)  

0
{0: 7.2}

1
{0: 7.2, 1: 29.333333333333332}

итоговый словарь {0: 7.2, 1: 29.333333333333332}


In [25]:
# импортируем класс StandardScaler, выполняющий стандартизацию
from sklearn.preprocessing import StandardScaler
# создаем модель стандартизации – экземпляр класса StandardScaler
standardscaler = StandardScaler()

In [26]:
# обучаем модель стандартизации, т.е. по каждому признаку 
# в обучающем массиве признаков вычисляем
# среднее значение признака и стандартное 
# отклонение признака для трансформации
standardscaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [27]:
# применяем модель стандартизации к обучающему массиву признаков: из исходного 
# значения признака вычитаем среднее значение признака, вычисленное 
# по ОБУЧАЮЩЕМУ массиву признаков, и результат делим на стандартное 
#отклонение признака, вычисленное по ОБУЧАЮЩЕМУ массиву признаков 
X_train_standardscaled = standardscaler.transform(X_train)

# применяем модель стандартизации к тестовому массиву признаков: из исходного 
# значения признака вычитаем среднее значение признака, вычисленное 
# по ОБУЧАЮЩЕМУ массиву признаков, и результат делим на стандартное 
# отклонение признака, вычисленное по ОБУЧАЮЩЕМУ массиву признаков 
X_test_standardscaled = standardscaler.transform(X_test)

In [28]:
# импортируем класс LogisticRegression,
# строящий логистическую регрессию
from sklearn.linear_model import LogisticRegression
# создаем модель логистической регрессии – экземпляр класса LogisticRegression
logreg = LogisticRegression(solver='lbfgs', max_iter=200)
# обучаем модель логистической регрессии, т.е. 
# находим параметры - регрессионные коэффициенты
logreg.fit(X_train_standardscaled, y_train)
# оцениваем качество модели на обучающих данных
print('Правильность на обучающей выборке: {:.3f}'.format(
    logreg.score(X_train_standardscaled, y_train)))
# оцениваем качество модели на тестовых данных
print('Правильность на тестовой выборке: {:.3f}'.format(
    logreg.score(X_test_standardscaled, y_test)))

Правильность на обучающей выборке: 0.900
Правильность на тестовой выборке: 0.900


In [29]:
# вычисляем спрогнозированные значения зависимой переменной
# для тестового массива признаков
logreg_predvalues = logreg.predict(X_test_standardscaled)
logreg_predvalues[:5]

array([0, 0, 0, 0, 0])

In [30]:
# вычисляем вероятности классов зависимой переменной
# для тестового массива признаков
logreg_probabilities = logreg.predict_proba(X_test_standardscaled)
logreg_probabilities[:5]

array([[0.91027855, 0.08972145],
       [0.89847518, 0.10152482],
       [0.88300257, 0.11699743],
       [0.90234211, 0.09765789],
       [0.92554507, 0.07445493]])

In [31]:
# создаем игрушечный обучающий датафрейм pandas
toy_train = pd.DataFrame({'Balance': [8.3, 9.4, 10.2, 0], 
                          'Age': [23, 29, 36, 44]})

# создаем игрушечный тестовый датафрейм pandas
toy_test = pd.DataFrame({'Balance': [10.4, 3.1, 22.5, -1], 
                         'Age': [13, 19, 66, 33]})

In [32]:
# создаем собственный класс, выполняющий стандартизацию
class CustomScaler(BaseEstimator, TransformerMixin):
    def __init__(self, copy=True, corr=False):
        self.copy = copy
        self.corr = corr
        
    def __is_numpy(self, X):
        # частный метод, который с помощью функции isinstance()
        # проверяет, является ли наш объект массивом NumPy
        return isinstance(X, np.ndarray)    
    
    def fit(self, X, y=None):
        self.mean_dict = {}
        self.std_dict = {}
        
        # если 1D-массив, то переводим в 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
             
        # записываем результат __is_numpy()
        is_np = self.__is_numpy(X)
        
        # если объект - массив NumPy, выполняем следующие действия:
        if is_np:
            # записываем количество столбцов
            ncols = X.shape[1]
            # по каждому столбцу массива NumPy
            for col in range(ncols):
                # вычисляем среднее
                self.mean_dict[col] = np.mean(X[:, col])
                # вычисляем стандартное отклонение
                self.std_dict[col] = np.std(X[:, col], ddof=1)
                if self.corr:
                    self.std_dict[col] = np.std(X[:, col], ddof=1) * 2
                
        else:
            # записываем список столбцов
            num_columns = X.select_dtypes(exclude='object').columns.tolist()
            # по каждому столбцу датафрейма pandas
            for col in num_columns:
                # вычисляем среднее
                self.mean_dict[col] = X[col].mean()
                # вычисляем стандартное отклонение
                self.std_dict[col] = X[col].std()
                if self.corr:
                    self.std_dict[col] = X[col].std() * 2
                
            
        return self
    
    def transform(self, X):
        
        if self.copy:
            X = X.copy()
            
        # если 1D-массив, то переводим в 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
        
        # записываем результат __is_numpy()
        is_np = self.__is_numpy(X)
            
        # если объект - массив NumPy, выполняем следующие действия:
        if is_np:
            # записываем количество столбцов
            ncols = X.shape[1]
            # по каждому столбцу массива NumPy
            for col in range(ncols):
                # выполняем стандартизацию
                X[:, col] = (X[:, col] - self.mean_dict[col]) / self.std_dict[col]
        
        else:
            # записываем список столбцов
            num_columns = X.select_dtypes(exclude='object').columns.tolist()
            # по каждому столбцу датафрейма pandas
            for col in num_columns:
                # выполняем стандартизацию
                X[col] = (X[col] - self.mean_dict[col]) / self.std_dict[col]
                
        return X

In [33]:
# смотрим, как должны выглядеть результаты
print((toy_train['Balance'] - toy_train['Balance'].mean()) / toy_train['Balance'].std())
print((toy_train['Age'] - toy_train['Age'].mean()) / toy_train['Age'].std())
print('')
print((toy_test['Balance'] - toy_train['Balance'].mean()) / toy_train['Balance'].std())
print((toy_test['Age'] - toy_train['Age'].mean()) / toy_train['Age'].std())

0    0.281031
1    0.514340
2    0.684019
3   -1.479390
Name: Balance, dtype: float64
0   -1.104315
1   -0.441726
2    0.331295
3    1.214747
Name: Age, dtype: float64

0    0.726439
1   -0.821883
2    3.292835
3   -1.691489
Name: Balance, dtype: float64
0   -2.208631
1   -1.546041
2    3.644240
3    0.000000
Name: Age, dtype: float64


In [34]:
# применяем класс к датафреймам pandas
scaler = CustomScaler()
scaler.fit(toy_train)
toy_train = scaler.transform(toy_train)
toy_test = scaler.transform(toy_test)

In [35]:
# смотрим результат на игрушечном обучающем датафрейме
toy_train

,Balance,Age
0,0.281031,-1.104315
1,0.514340,-0.441726
2,0.684019,0.331295
3,-1.479390,1.214747


In [36]:
# взглянем на стандартные отклонения
for col in toy_train.columns:
    print(col, toy_train[col].std())

Balance 1.0
Age 1.0


In [37]:
# смотрим результат на игрушечном тестовом датафрейме
toy_test

,Balance,Age
0,0.726439,-2.208631
1,-0.821883,-1.546041
2,3.292835,3.644240
3,-1.691489,0.000000


In [38]:
# создаем игрушечный обучающий массив NumPy
np_toy_train = np.array(pd.DataFrame({'Balance': [8.3, 9.4, 10.2, 0], 
                                      'Age': [23, 29, 36, 44]}))

# создаем игрушечный тестовый массив NumPy
np_toy_test = np.array(pd.DataFrame({'Balance': [10.4, 3.1, 22.5, -1], 
                                     'Age': [13, 19, 66, 33]}))

In [39]:
# применяем класс к массивам NumPy
scaler = CustomScaler()
scaler.fit(np_toy_train)
np_toy_train = scaler.transform(np_toy_train)
np_toy_test = scaler.transform(np_toy_test)

In [40]:
# смотрим результат на игрушечном 
# обучающем массиве NumPy
np_toy_train

array([[ 0.28103104, -1.10431526],
       [ 0.51433982, -0.4417261 ],
       [ 0.68401894,  0.33129458],
       [-1.4793898 ,  1.21474679]])

In [41]:
# смотрим результат на игрушечном 
# тестовом массиве NumPy
np_toy_test

array([[ 0.72643872, -2.20863052],
       [-0.82188322, -1.54604137],
       [ 3.29283537,  3.64424036],
       [-1.6914887 ,  0.        ]])

In [42]:
# создаем игрушечный обучающий датафрейм pandas
toy_train = pd.DataFrame({'Balance': [8.3, 9.4, 10.2, 0], 
                          'Age': [23, 29, 36, 44]})

# создаем игрушечный тестовый датафрейм pandas
toy_test = pd.DataFrame({'Balance': [10.4, 3.1, 22.5, -1], 
                         'Age': [13, 19, 66, 33]})

In [43]:
# смотрим, как должны выглядеть результаты
print((toy_train['Balance'] - toy_train['Balance'].mean()) / (toy_train['Balance'].std() * 2))
print((toy_train['Age'] - toy_train['Age'].mean()) / (toy_train['Age'].std() * 2))
print('')
print((toy_test['Balance'] - toy_train['Balance'].mean()) / (toy_train['Balance'].std() * 2))
print((toy_test['Age'] - toy_train['Age'].mean()) / (toy_train['Age'].std() * 2))

0    0.140516
1    0.257170
2    0.342009
3   -0.739695
Name: Balance, dtype: float64
0   -0.552158
1   -0.220863
2    0.165647
3    0.607373
Name: Age, dtype: float64

0    0.363219
1   -0.410942
2    1.646418
3   -0.845744
Name: Balance, dtype: float64
0   -1.104315
1   -0.773021
2    1.822120
3    0.000000
Name: Age, dtype: float64


In [44]:
# применяем класс к датафреймам pandas
scaler = CustomScaler(corr=True)
scaler.fit(toy_train)
toy_train = scaler.transform(toy_train)
toy_test = scaler.transform(toy_test)

# смотрим результат на игрушечном обучающем датафрейме
toy_train

,Balance,Age
0,0.140516,-0.552158
1,0.257170,-0.220863
2,0.342009,0.165647
3,-0.739695,0.607373


In [45]:
# взглянем на стандартные отклонения
for col in toy_train.columns:
    print(col, toy_train[col].std())

Balance 0.5
Age 0.5


In [46]:
# смотрим результат на игрушечном тестовом датафрейме
toy_test

,Balance,Age
0,0.363219,-1.104315
1,-0.410942,-0.773021
2,1.646418,1.822120
3,-0.845744,0.000000


In [47]:
X = np.random.uniform(low=100, high=130, size=1000000)
X_numpy = X.reshape([10000,100])
X_pandas = pd.DataFrame(X_numpy)
scaler = CustomScaler()

In [48]:
%timeit scaler.fit(X_numpy)

10.2 ms ± 804 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
%timeit scaler.transform(X_numpy)

7.27 ms ± 323 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
import numba as nb

In [51]:
from numba import jit

In [52]:
@nb.jit(nopython=True)
def nb_mean(arr):
    return np.sum(arr) / len(arr)

In [53]:
@nb.jit(nopython=True)
def nb_std(arr):
    l = arr.shape[0]
    mean = nb_mean(arr)
    sumsq = 0.0
    for i in range(l):
        sumsq += (arr[i] - mean) ** 2
    result = np.sqrt(sumsq / (l - 1))
    return result    

In [54]:
arr = np.array([2.0, 4.0, 6.0])

In [55]:
np.std(arr, ddof=1)

2.0

In [56]:
nb_std(arr)

2.0

In [57]:
# создаем собственный класс, выполняющий стандартизацию
class CustomScalerNumba(BaseEstimator, TransformerMixin):
    def __init__(self, copy=True, corr=False):
        self.copy = copy
        self.corr = corr
        
    def __is_numpy(self, X):
        # частный метод, который с помощью функции isinstance()
        # проверяет, является ли наш объект массивом NumPy
        return isinstance(X, np.ndarray)    
    
    def fit(self, X, y=None):
        self.mean_dict = {}
        self.std_dict = {}
        
        # если 1D-массив, то переводим в 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
             
        # записываем результат __is_numpy()
        is_np = self.__is_numpy(X)
        
        # если объект - массив NumPy, выполняем следующие действия:
        if is_np:
            # записываем количество столбцов
            ncols = X.shape[1]
            # по каждому столбцу массива NumPy
            for col in range(ncols):
                # вычисляем среднее
                self.mean_dict[col] = nb_mean(X[:, col])
                # вычисляем стандартное отклонение
                self.std_dict[col] = nb_std(X[:, col])
                if self.corr:
                    self.std_dict[col] = nb_std(X[:, col]) * 2
                
        else:
            # записываем список столбцов
            num_columns = dtypes[X.dtypes != 'object'].index
            # по каждому столбцу датафрейма pandas
            for col in num_columns:
                # вычисляем среднее
                self.mean_dict[col] = X[col].mean()
                # вычисляем стандартное отклонение
                self.std_dict[col] = X[col].std()
                if self.corr:
                    self.std_dict[col] = X[col].std() * 2
                
            
        return self
    
    def transform(self, X):
        
        if self.copy:
            X = X.copy()
            
        # если 1D-массив, то переводим в 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
        
        # записываем результат __is_numpy()
        is_np = self.__is_numpy(X)
            
        # если объект - массив NumPy, выполняем следующие действия:
        if is_np:
            # записываем количество столбцов
            ncols = X.shape[1]
            # по каждому столбцу массива NumPy
            
            for col in range(ncols):
                # выполняем стандартизацию
                X[:, col] = (X[:, col] - self.mean_dict[col]) / self.std_dict[col]
        
        else:
            # записываем список столбцов
            num_columns = dtypes[X.dtypes != 'object'].index
            # по каждому столбцу датафрейма pandas
            for col in num_columns:
                # выполняем стандартизацию
                X[col] = (X[col] - self.mean_dict[col]) / self.std_dict[col]
                
        return X

In [58]:
numbascaler = CustomScalerNumba()

In [59]:
%timeit numbascaler.fit(X_numpy)

4.47 ms ± 147 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [60]:
%timeit numbascaler.transform(X_numpy)

7.36 ms ± 219 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [61]:
# создаем игрушечный обучающий массив NumPy
np_toy_train = np.array(pd.DataFrame({'Balance': [8.3, 9.4, 10.2, 0], 
                                      'Age': [23, 29, 36, 44]}))

# создаем игрушечный тестовый массив NumPy
np_toy_test = np.array(pd.DataFrame({'Balance': [10.4, 3.1, 22.5, -1], 
                                     'Age': [13, 19, 66, 33]}))

In [62]:
# применяем класс к массивам NumPy
numbascaler.fit(np_toy_train)
np_toy_train = numbascaler.transform(np_toy_train)
np_toy_test = numbascaler.transform(np_toy_test)

In [63]:
np_toy_train

array([[ 0.28103104, -1.10431526],
       [ 0.51433982, -0.4417261 ],
       [ 0.68401894,  0.33129458],
       [-1.4793898 ,  1.21474679]])

In [64]:
np_toy_test

array([[ 0.72643872, -2.20863052],
       [-0.82188322, -1.54604137],
       [ 3.29283537,  3.64424036],
       [-1.6914887 ,  0.        ]])

In [65]:
class StepanStandartizator:

    def __init__(self, copy=True):
        self._standardizer_dict = {}
        self.copy = copy

    @staticmethod
    def __is_numpy(x):
        return isinstance(x, np.ndarray)

    def fit(self, x, y=None):

        # Пустой список, который будет заполняться парами выборочных средних и 
        # несмещенных выборочных дисперсий
        # Флаг массива Numpy
        is_np = self.__is_numpy(x)

        # Гарантия двумерного массива
        if len(x.shape) == 1:
            x = x.reshape(-1, 1)

        # Число столбцов
        ncols = x.shape[1]

        # Ветвление, возвращает пару выборочное среднее - несмещенная дисперсия 
        # Для numpy.ndarray или pd.DataFrame
        if is_np:
            for col in range(ncols):
                self._standardizer_dict[col] = [np.mean(x[:, col]), np.nanvar(x[:, col], ddof=0)]
        else:
            for col in x.columns:
                self._standardizer_dict[col] = [x[col].mean(), x[col].var()]

        return self

    def transform(self, x):

        if self.copy:
            x = x.copy()

        is_np = self.__is_numpy(x)

        # Гарантия двумерного массива
        if len(x.shape) == 1:
            x = x.reshape(-1, 1)

        nrows = x.shape[0]
        # Число столбцов
        ncols = x.shape[1]
        if is_np:
            for col in range(ncols):
                for row in range(nrows):
                    x[row, col] = (x[row, col] - self._standardizer_dict[col][0]) / np.sqrt(
                        self._standardizer_dict[col][1])
        else:
            for col in x.columns:
                for row in range(nrows):
                    x.iloc[row, col] = (x.iloc[row, col] - self._standardizer_dict[col][0]) / np.sqrt(
                        self._standardizer_dict[col][1])

        return x

In [66]:
# создаем игрушечный обучающий массив NumPy
np_toy_train = np.array(pd.DataFrame({'Balance': [8.3, 9.4, 10.2, 0], 
                                      'Age': [23, 29, 36, 44]}))

# создаем игрушечный тестовый массив NumPy
np_toy_test = np.array(pd.DataFrame({'Balance': [10.4, 3.1, 22.5, -1], 
                                     'Age': [13, 19, 66, 33]}))

In [67]:
# применяем класс к массивам NumPy
stepanscaler = StepanStandartizator()
stepanscaler.fit(np_toy_train)
np_toy_train = stepanscaler.transform(np_toy_train)
np_toy_test = stepanscaler.transform(np_toy_test)

In [68]:
# смотрим результат на игрушечном 
# обучающем массиве NumPy
np_toy_train

array([[ 0.32450669, -1.27515343],
       [ 0.59390847, -0.51006137],
       [ 0.78983704,  0.38254603],
       [-1.7082522 ,  1.40266877]])

In [69]:
np_toy_test

array([[ 0.83881918, -2.55030685],
       [-0.949029  , -1.7852148 ],
       [ 3.80223877,  4.20800631],
       [-1.95316291,  0.        ]])